# RO-MPI validation

## Imports

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns
import xarray as xr
import warnings
import tqdm
import pathlib
import cmocean
import os

# Import custom modules
import src.utils
from src.XRO import XRO, xcorr

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## get filepaths
DATA_FP = pathlib.Path(os.environ["DATA_FP"])
SAVE_FP = pathlib.Path(os.environ["SAVE_FP"])

## Functions

In [2]:
def fit_RO_to_ensemble(
    model, Th_data, T_var, h_var, fit_to_members=False, **fit_kwargs
):
    """Fit RO parameters to ensemble"""

    ## fit model and generate synthetic ensemble
    if fit_to_members:

        ## fit to individual ensemble members
        kwargs = dict(model=model, T_var=T_var, h_var=h_var, verbose=True)
        _, params = src.utils.get_RO_ensemble(Th_data, **kwargs)

    else:

        ## fit to all ensemble members simultaneously
        params = model.fit_matrix(
            Th_data[[T_var, h_var]], **fit_kwargs
        )  # ac_mask_idx=ac_mask_idx, maskNT=[])

    return params


def generate_RO_ensemble(model, params, fit_to_members=False, **simulation_kwargs):
    """generate RO ensemble using given model and parameters"""

    if fit_to_members:

        RO_ensemble = src.utils.generate_ensemble(model, params, **simulation_kwargs)

    else:

        RO_ensemble = model.simulate(fit_ds=params, **simulation_kwargs)

    return RO_ensemble

## Load data

In [3]:
## MPI data
mpi_load_fp = pathlib.Path(DATA_FP, "mpi_Th", "Th.nc")
Th = xr.open_dataset(mpi_load_fp)

## Fit RO to ensemble data

In [4]:
## specify variables to use in RO fit order of annual cycle
T_var = "T_34"
h_var = "h"
ac_order = 3

## specify whether to fit parameters to each ens. member individually
## if False, fit single set of parameters to all ensemble members
fit_to_members = False


### specify fit kwargs
## ac_mask_idx: which indices to mask annual cycle out for.
## maskNT: which nonlinear terms to include for dT/dt
fit_kwargs = dict(ac_mask_idx=[(1, 1)], maskNT=[])

## get data for early/late period
Th_early = Th.isel(time=slice(None, 588))
Th_late = Th.isel(time=slice(-588, None))

## specify model to use
model = XRO(ncycle=12, ac_order=ac_order, is_forward=True)

## fit models to early/late periods
RO_params_early = fit_RO_to_ensemble(
    model,
    Th_data=Th_early,
    T_var=T_var,
    h_var=h_var,
    fit_to_members=False,
    **fit_kwargs
)
RO_params_late = fit_RO_to_ensemble(
    model, Th_data=Th_late, T_var=T_var, h_var=h_var, fit_to_members=False, **fit_kwargs
)

/Users/theo/research/enso2025_xro/envs/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: invalid value encountered in power
  result_data = func(*input_data)
/Users/theo/research/enso2025_xro/envs/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: invalid value encountered in power
  result_data = func(*input_data)
/Users/theo/research/enso2025_xro/envs/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: invalid value encountered in power
  result_data = func(*input_data)
/Users/theo/research/enso2025_xro/envs/lib/python3.13/site-packages/xarray/computation/apply_ufunc.py:821: RuntimeWarning: invalid value encountered in power
  result_data = func(*input_data)


## Get ensemble of RO simulations

In [5]:
## specify arguments for simulation
simulation_kwargs = dict(
    nyear=49, ncopy=1000, seed=1000, X0_ds=Th[[T_var, h_var]].isel(member=0, time=0)
)

## perform simulations
RO_ensemble_early = generate_RO_ensemble(
    model, RO_params_early, fit_to_members=fit_to_members, **simulation_kwargs
)
RO_ensemble_late = generate_RO_ensemble(
    model, RO_params_late, fit_to_members=fit_to_members, **simulation_kwargs
)

## filter outsimulations which blow up
# bad_sim = (RO_ensemble[T_var] > 5).any("time")
# RO_ensemble = RO_ensemble.isel(member=~bad_sim)

## Diagnostics

### Seasonality

In [6]:
def plot_seasonal_cyc(ax, x, varname, **plot_kwargs):
    """print seasonal cycle of data in x on specified ax object"""

    ## func to compute stats
    get_std = lambda x: x.groupby("time.month").std("time")
    get_stats = lambda x: src.utils.get_ensemble_stats(get_std(x[varname]))

    ## compute std for each dataset
    x_plot = get_stats(x)

    ## months (x-coordinate for plotting
    months = np.arange(1, 13)

    ## plot MPI ensemble mean
    plot_data = ax.plot(np.arange(1, 13), x_plot.sel(posn="center"), **plot_kwargs)

    ## plot MPI bounds
    kwargs = dict(c=plot_data[0].get_color(), ls="--", lw=1)
    for bound in ["upper", "lower"]:
        ax.plot(months, x_plot.sel(posn=bound), **kwargs)

    return plot_data


def plot_seasonal_comp(
    ax, x0, x1, varname="T_34", plot_kwargs0=dict(), plot_kwargs1=dict()
):
    """Plot comparison of seasonal cycle between 2 ensembles"""

    ## plot data
    plot_data0 = plot_seasonal_cyc(ax, x0, varname=varname, **plot_kwargs0)
    plot_data1 = plot_seasonal_cyc(ax, x1, varname=varname, **plot_kwargs1)

    ## format ax
    ax.set_xticks([1, 8, 12], labels=["Jan", "Aug", "Dec"])
    ax.set_xlabel("Month")
    ax.set_ylim([0.4, 1.2])
    ax.set_yticks([])

    return plot_data0, plot_data1


def format_axs(axs):
    """add formatting to paneled subplot of seasonal cycle"""

    axs[0].set_yticks([0.5, 1])
    axs[0].set_ylabel(r"$^{\circ}$C")
    axs[0].set_title(r"1850-1889")
    axs[1].set_title(r"2051-2100")
    axs[2].set_title(r"Early vs. Late (MPI)")
    axs[2].set_yticks([0.5, 1])
    axs[2].yaxis.set_ticks_position("right")
    axs[0].legend(prop=dict(size=8))
    axs[1].legend(prop=dict(size=8))
    axs[2].legend(prop=dict(size=8))
    return

In [7]:
### Set up plot
fig, axs = plt.subplots(1, 3, figsize=(7, 2), layout="constrained")

## plot MPI vs. RO (early period)
plot_data_early = plot_seasonal_comp(
    axs[0],
    x0=Th_early,
    x1=RO_ensemble_early,
    plot_kwargs0=dict(label="MPI"),
    plot_kwargs1=dict(label="RO"),
)

## Plot MPI vs RO (late period)
plot_data_late = plot_seasonal_comp(
    axs[1],
    x0=Th_late,
    x1=RO_ensemble_late,
    plot_kwargs0=dict(label="MPI"),
    plot_kwargs1=dict(label="RO"),
)

## Plot early vs late (MPI)
plot_data_delta = plot_seasonal_comp(
    axs[2],
    x0=RO_ensemble_early,
    x1=RO_ensemble_late,
    plot_kwargs0=dict(label="1850-1889", c="k"),
    plot_kwargs1=dict(label="2051-2100", c="r"),
)

## label plot
format_axs(axs)

plt.show()

### Power spectrum
To-do...

### $T$, $h$ cross-correlation

Make plots

In [8]:
def format_axs(axs):
    """make axes nice for plot"""

    for ax in axs[1, :]:
        ax.set_xticks([-12, 0, 12], labels=[-1, 0, 1])
        ax.set_xlabel("Lag (years)")
    for ax in axs[:, 0]:
        ax.set_yticks([0, 0.5, 1])
        ax.set_ylabel("Corr.")
    axs[0, 0].set_title(r"1850-1889")
    axs[0, 1].set_title(r"2051-2100")
    axs[0, 2].set_title(r"Early vs. Late (MPI)")
    return


def plot_xcorr(ax, data, **plot_kwargs):
    """plot mean and bounds for data"""

    ## center
    plot_data = ax.plot(data.lag, data.sel(posn="center"), **plot_kwargs)

    ## bounds
    ax.fill_between(
        data.lag,
        data.sel(posn="upper"),
        data.sel(posn="lower"),
        color=plot_data[0].get_color(),
        alpha=0.2,
    )

    ## set axis specs
    ax.set_ylim([-0.5, 1.05])
    ax.set_xlim([-18, 18])
    ax.set_xticks([])
    ax.set_yticks([])
    axis_kwargs = dict(c="k", lw=0.5, alpha=0.5)
    ax.axhline(0, **axis_kwargs)
    ax.axvline(0, **axis_kwargs)

    return


def plot_xcorr2(axs, x, T_var="T_34", h_var="h", **plot_kwargs):
    """plot <T,T> and <h,T> correlation on set of axs"""

    ## compute cross correlation and get ensemble stats
    xcorr_eval = xcorr(x, x[T_var], maxlags=18)
    xcorr_stats = src.utils.get_ensemble_stats(xcorr_eval)

    ## plot autocorrelation/cross correlation on separate axes
    plot_xcorr(axs[0], xcorr_stats[T_var], **plot_kwargs)
    plot_xcorr(axs[1], xcorr_stats[h_var], **plot_kwargs)

    return


fig, axs = plt.subplots(2, 3, figsize=(7, 4.5), layout="constrained")

plot_xcorr2(axs[:, 0], Th_early)
plot_xcorr2(axs[:, 0], RO_ensemble_early)

plot_xcorr2(axs[:, 1], Th_late)
plot_xcorr2(axs[:, 1], RO_ensemble_late)

plot_xcorr2(axs[:, 2], Th_early, c="k")
plot_xcorr2(axs[:, 2], Th_late, c="r")

format_axs(axs)